In [1]:
%load_ext autoreload
%autoreload 2
from benchmark.utils import *
from benchmark.preprocess import *
from benchmark.constants import *
from benchmark.db_import import *
from benchmark.ground_truth import *
from benchmark.construct_threatrace_graph import *
from benchmark.threatrace import *
#from benchmark import sql_api
import logging 
from torch_geometric import seed_everything
logging.basicConfig(
    format='%(asctime)s - %(levelname)s - %(message)s',
    level=logging.INFO,
    filename='app.log',
    #filemode='w'
    filemode='a',
)
logging.info('This will get logged to a file')

2023-06-08 12:05:00


In [2]:
seed_everything(1234)

In [3]:
import configparser
config_dir = "/Users/robinbuchta/Documents/GitHub/trustathsh/projekte/secder/Fachliches/darpa_benchmark_lite_sql/configs"
config_file_record_03 = config_dir + '/eng3/cadets/cadets_03_record.ini'
config_rec3 = configparser.ConfigParser()
config_rec3.read(config_file_record_03)


['/Users/robinbuchta/Documents/GitHub/trustathsh/projekte/secder/Fachliches/darpa_benchmark_lite_sql/configs/eng3/cadets/cadets_03_record.ini']

In [4]:
# import configparser
# config_dir = "/Users/robinbuchta/Documents/GitHub/trustathsh/projekte/secder/Fachliches/darpa_benchmark_lite_sql/configs"
# config_file_record_01 = config_dir + '/eng3/cadets/cadets_01_record.ini'
# config_rec1 = configparser.ConfigParser()
# config_rec1.read(config_file_record_01)


In [5]:
# mypygdata_rec1 = MyPyGDataset(config_rec1, own_timestamp_percent=15, extension="15percentTrain") # 14 min 

In [6]:
def print_data_stats(data):
    print()
    print(data)
    print('===========================================================================================================')

    # Gather some statistics about the graph.
    print(f'Number of nodes: {data.num_nodes}')
    print(f'Number of edges: {data.num_edges}')
    print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
    print(f'Number of training nodes: {data.train_mask.sum()}')
    print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
    print(f'Has isolated nodes: {data.has_isolated_nodes()}')
    print(f'Has self-loops: {data.has_self_loops()}')
    print(f'Is undirected: {data.is_undirected()}')

In [7]:
mypygdata = MyPyGDataset(config_rec3, own_timestamp_percent=15, extension="15percentTrain") # 4min

In [8]:
data_train = mypygdata[0][0]
data_test = mypygdata[0][1]
print(data_train)
print(data_test)

Data(x=[125787, 56], edge_index=[2, 1426653], y=[125787], train_mask=[125787], test_mask=[125787])
Data(x=[574338, 56], edge_index=[2, 7375768], y=[574338], train_mask=[574338], test_mask=[574338])


In [9]:
#runtime: 2min 20s
# get Neighbors of atk nodes
atk_nodes = get_atk_nodes(config_rec3)
mapped_atk_nodes = find_mapping_value_for_gt_uuid(config_rec3, atk_nodes)
print(len(mapped_atk_nodes))
mapped_atk_nodes_decremented = [x - 1 for x in mapped_atk_nodes]
from torch_geometric.utils import k_hop_subgraph

mapped_atk_nodes_decremented_with_2_hop, edge_index, mapping, edge_mask = k_hop_subgraph(mapped_atk_nodes_decremented, num_hops=2, edge_index=data_test.edge_index)
print(len(mapped_atk_nodes_decremented_with_2_hop))

File exists.
SELECT id, node FROM cadets_r03_tt_nodes_test WHERE node IN ('5807443D-3F5F-11E8-A5CB-3FA3753A265A','45D0B6A8-3F2A-11E8-A5CB-3FA3753A265A','0C650D3B-CFDE-594D-9F8F-8D20DE590C1C','67FA18C3-6008-5AEA-B84E-E156CB44DDC3','E39AE419-00B8-5064-9732-70EF1D9E7DAA','C5DFC359-3F59-11E8-A5CB-3FA3753A265A','742CFF08-3F42-11E8-A5CB-3FA3753A265A','00BBE0B4-BA58-5011-A8F5-9F92E8CF8F1C','0649284E-3E2E-11E8-A5CB-3FA3753A265A','B5F044FC-624D-510B-9DA4-6E3327387877','0A8A96FE-CCAC-5A59-99A4-ACD28026F656','0E781417-A5F8-578D-812F-EA4628E44F8B','266447B6-3EC6-11E8-A5CB-3FA3753A265A','54DBB08C-3EA7-11E8-A5CB-3FA3753A265A','9779E4E0-3F59-11E8-A5CB-3FA3753A265A','3936F432-3E9A-11E8-A5CB-3FA3753A265A','EBF7FF29-3EB7-11E8-A5CB-3FA3753A265A','70152050-61C4-5FB4-9B0C-365CBA5896EC','ADEDF311-8C7B-5074-AE17-9913D6EB30A4','8E64012A-3E94-11E8-A5CB-3FA3753A265A','18D0EF0A-3F35-11E8-A5CB-3FA3753A265A','03B2D7CC-3DDA-11E8-A5CB-3FA3753A265A','F980133D-41DB-5029-B5F1-508AA3F083D5','C7178515-1444-5E7C-A2B1-5E5A

TypeError: unsupported operand type(s) for -: 'set' and 'int'

In [ ]:
print_data_stats(data_train) # Number of nodes: 950774 Number of edges: 12192401 # 10% =  Number of nodes: 127452 Number of edges: 1261612
print_data_stats(data_test) # Number of nodes: 942812 Number of edges: 12082603
data_test.n_id = torch.arange(data_test.num_nodes)
data_train.n_id = torch.arange(data_train.num_nodes)


Data(x=[125787, 56], edge_index=[2, 1426653], y=[125787], train_mask=[125787], test_mask=[125787])
Number of nodes: 125787
Number of edges: 1426653
Average node degree: 11.34
Number of training nodes: 125787
Training node label rate: 1.00
Has isolated nodes: True
Has self-loops: True
Is undirected: False

Data(x=[574338, 56], edge_index=[2, 7375768], y=[574338], train_mask=[574338], test_mask=[574338])
Number of nodes: 574338
Number of edges: 7375768
Average node degree: 12.84
Number of training nodes: 574338
Training node label rate: 1.00
Has isolated nodes: True
Has self-loops: True
Is undirected: False


In [ ]:
print(f"Unique value count: {data_test.test_mask.unique(return_counts=True)}")
print(f"Unique value count: {data_test.train_mask.unique(return_counts=True)}")

Unique value count: (tensor([True]), tensor([574338]))
Unique value count: (tensor([True]), tensor([574338]))


In [19]:
tt_data_train = create_tt_orig_data_train()
tt_data_test = create_tt_orig_data_test()

2023-06-08 14:11:07


In [20]:
print_data_stats(tt_data_train) # Number of nodes: 950774 Number of edges: 12192401 # 10% =  Number of nodes: 127452 Number of edges: 1261612
print_data_stats(tt_data_test) # Number of nodes: 942812 Number of edges: 12082603
data_test.n_id = torch.arange(tt_data_test.num_nodes)
data_train.n_id = torch.arange(tt_data_train.num_nodes)


Data(x=[362645, 56], edge_index=[2, 4347080], y=[362645], train_mask=[362645], test_mask=[362645], n_id=[362645])
Number of nodes: 362645
Number of edges: 4347080
Average node degree: 11.99
Number of training nodes: 362645
Training node label rate: 1.00
Has isolated nodes: True
Has self-loops: True
Is undirected: False

Data(x=[357174, 56], edge_index=[2, 4316489], y=[357174], train_mask=[357174], test_mask=[357174], n_id=[357174])
Number of nodes: 357174
Number of edges: 4316489
Average node degree: 12.09
Number of training nodes: 357174
Training node label rate: 1.00
Has isolated nodes: True
Has self-loops: True
Is undirected: False


In [13]:
#ttp = ThreaTracePipeline(config_rec3, data_train, data_test)
ttp = ThreaTracePipeline(config_rec3, tt_data_train, tt_data_test)

In [14]:
ttp.delete_old_models()

In [15]:
ttp.pretraining()

1 1907.0634214064135 0.8703553061534007 2023-06-08 12:05:42
2 254.62111721873245 0.9256490507245377 2023-06-08 12:05:43
3 73.96026791962028 0.9434653724717009 2023-06-08 12:05:45
4 69.66836246161283 0.9444332611782873 2023-06-08 12:05:46
5 26.64936204065674 0.951622109776779 2023-06-08 12:05:47
6 44.589029803017226 0.9583669980283749 2023-06-08 12:05:48
7 36.425938945111845 0.958689627597237 2023-06-08 12:05:49
8 31.22865186210443 0.9612403314536254 2023-06-08 12:05:50
9 24.166916585080003 0.9617228970480773 2023-06-08 12:05:51
10 35.4306108990555 0.9550690068800066 2023-06-08 12:05:52
11 25.427804775747163 0.9597457568696659 2023-06-08 12:05:53
12 22.81969082704355 0.9640171517599857 2023-06-08 12:05:54
13 21.031900042430973 0.965018130678763 2023-06-08 12:05:55
14 21.05077376098526 0.9660880475396049 2023-06-08 12:05:56
15 23.284661996819707 0.9667884570309807 2023-06-08 12:05:57
16 21.406967654547927 0.9669180603620621 2023-06-08 12:05:58
17 16.44456515966236 0.9698355140702339 2023

In [16]:
ttp.multi_model_training()

New Round
false_classified, true_classified
14612 348033
Model saved loop_num: 0 2023-06-08 12:06:26
New Round
false_classified, true_classified
4644 9968
Model saved loop_num: 1 2023-06-08 12:06:34
New Round
false_classified, true_classified
2414 2230
Model saved loop_num: 2 2023-06-08 12:06:42
New Round
false_classified, true_classified
745 1669
Model saved loop_num: 3 2023-06-08 12:06:44
New Round
false_classified, true_classified
439 306
Model saved loop_num: 4 2023-06-08 12:06:45
New Round
false_classified, true_classified
225 214
Model saved loop_num: 5 2023-06-08 12:06:45
New Round
false_classified, true_classified
61 164
Model saved loop_num: 6 2023-06-08 12:06:46
New Round
false_classified, true_classified
39 22
Model saved loop_num: 7 2023-06-08 12:06:46
New Round
false_classified, true_classified
30 9
Model saved loop_num: 8 2023-06-08 12:06:46
New Round
false_classified, true_classified
15 15
Model saved loop_num: 9 2023-06-08 12:06:46
New Round
false_classified, true_class

In [17]:
#tt gt
# read txt file row by row and fill list with values
file_path = "/Users/robinbuchta/Documents/GitHub/threaTrace/groundtruth/cadets.txt"
def read_txt_file(file_path):
    with open(file_path, 'r') as file:
        data = file.read().splitlines()
    return data
tt_gt_uuids = read_txt_file(file_path)
print(len(tt_gt_uuids))
tt_mapped_atk_nodes = find_mapping_value_for_gt_uuid(config_rec3, tt_gt_uuids)
tt_mapped_atk_nodes_decremented = [x - 1 for x in tt_mapped_atk_nodes]


12858
SELECT id, node FROM cadets_r03_tt_nodes_test WHERE node IN ('9EF37E2E-3E80-11E8-A5CB-3FA3753A265A','A4DD7C60-3E80-11E8-A5CB-3FA3753A265A','A55A32F5-3E80-11E8-A5CB-3FA3753A265A','878A0030-3E80-11E8-A5CB-3FA3753A265A','8145E23B-3E80-11E8-A5CB-3FA3753A265A','A54E7444-3E80-11E8-A5CB-3FA3753A265A','A69A88DD-3E80-11E8-A5CB-3FA3753A265A','80265456-3E80-11E8-A5CB-3FA3753A265A','7172780D-3E80-11E8-A5CB-3FA3753A265A','875F1C7B-3E80-11E8-A5CB-3FA3753A265A','797A71AF-3E80-11E8-A5CB-3FA3753A265A','89268999-3E80-11E8-A5CB-3FA3753A265A','89CE2920-3E80-11E8-A5CB-3FA3753A265A','9F1FABF0-3E80-11E8-A5CB-3FA3753A265A','A547982E-3E80-11E8-A5CB-3FA3753A265A','792914DC-3E80-11E8-A5CB-3FA3753A265A','80629A3A-3E80-11E8-A5CB-3FA3753A265A','89D227A2-3E80-11E8-A5CB-3FA3753A265A','A6654946-3E80-11E8-A5CB-3FA3753A265A','7A72825A-3E80-11E8-A5CB-3FA3753A265A','9EDFF511-3E80-11E8-A5CB-3FA3753A265A','71E1962F-3E80-11E8-A5CB-3FA3753A265A','876EFAB3-3E80-11E8-A5CB-3FA3753A265A','9EECA0F5-3E80-11E8-A5CB-3FA3753A265

TypeError: unsupported operand type(s) for -: 'set' and 'int'

In [ ]:
len(tt_gt_uuids)

12858

In [ ]:
#tt_mapped_atk_nodes = find_mapping_value_for_gt_uuid(config_rec3, tt_gt_uuids)
tt_mapped_atk_nodes, translation = find_mapping_value_for_gt_uuid(config_rec3, tt_gt_uuids)

SELECT id, node FROM cadets_r03_tt_nodes_test WHERE node IN ('9EF37E2E-3E80-11E8-A5CB-3FA3753A265A','A4DD7C60-3E80-11E8-A5CB-3FA3753A265A','A55A32F5-3E80-11E8-A5CB-3FA3753A265A','878A0030-3E80-11E8-A5CB-3FA3753A265A','8145E23B-3E80-11E8-A5CB-3FA3753A265A','A54E7444-3E80-11E8-A5CB-3FA3753A265A','A69A88DD-3E80-11E8-A5CB-3FA3753A265A','80265456-3E80-11E8-A5CB-3FA3753A265A','7172780D-3E80-11E8-A5CB-3FA3753A265A','875F1C7B-3E80-11E8-A5CB-3FA3753A265A','797A71AF-3E80-11E8-A5CB-3FA3753A265A','89268999-3E80-11E8-A5CB-3FA3753A265A','89CE2920-3E80-11E8-A5CB-3FA3753A265A','9F1FABF0-3E80-11E8-A5CB-3FA3753A265A','A547982E-3E80-11E8-A5CB-3FA3753A265A','792914DC-3E80-11E8-A5CB-3FA3753A265A','80629A3A-3E80-11E8-A5CB-3FA3753A265A','89D227A2-3E80-11E8-A5CB-3FA3753A265A','A6654946-3E80-11E8-A5CB-3FA3753A265A','7A72825A-3E80-11E8-A5CB-3FA3753A265A','9EDFF511-3E80-11E8-A5CB-3FA3753A265A','71E1962F-3E80-11E8-A5CB-3FA3753A265A','876EFAB3-3E80-11E8-A5CB-3FA3753A265A','9EECA0F5-3E80-11E8-A5CB-3FA3753A265A','A6

In [ ]:
type(tt_gt_uuids)

list

In [ ]:
matching_elements = translation.intersection(tt_gt_uuids)

In [ ]:
len(matching_elements)

12857

In [ ]:
tt_mapped_atk_nodes_decremented = [x - 1 for x in tt_mapped_atk_nodes]

In [ ]:
#gt = mapped_atk_nodes_decremented_with_2_hop
#gt = mapped_atk_nodes_decremented
#gt = tt_mapped_atk_nodes_decremented
ttp.evaluation(tt_mapped_atk_nodes_decremented) # TP: 208, FP: 15, TN: 549868, FN: 24247
#TP: 39, FP: 5427, TN: 338895, FN: 12813 whole training 
#TP: 39, FP: 5367, TN: 338955, FN: 12813 nur ein model 
#TP: 10, FP: 1549, TN: 342773, FN: 12842
#TP: 46, FP: 57, TN: 344265, FN: 12806 with only one model somit besser als mit multi-model setup

Loop_num: 0  Accuracy:0.9341  true_classified:333653  false_classified:23521
hits:  23521
(tensor([0, 1, 2, 3, 4, 5]), tensor([3280, 8467, 2775, 3996, 4923,   80]))
start
{'tn': 357174}
i:  485308
i:  485535
i:  485841
i:  486588
i:  487173
gt
{'fn': 12852, 'tn': 344322}
{'fn': 12806, 'fp': 57, 'tn': 344265, 'tp': 46}
357174
46 57 344265 12806
TP: 46, FP: 57, TN: 344265, FN: 12806
Precision:  0.4466019417471392
Recall:  0.003579209461562375
F-Score:  0.00710150520876591
Accuracy:  0.963986740356241


In [23]:
# Get the shape of the tensor
rows, cols = data_test.x.shape
# Iterate over columns
for col in range(cols):
    unique_values, counts = torch.unique(data_test.x[:, col], return_counts=True)
    print(f"Column {col+1}:")
    for value, count in zip(unique_values, counts): 
        print(f"  Value: {value.item()}, Count: {count.item()}")

Column 1:
  Value: 0.0, Count: 573253
  Value: 2.0, Count: 379
  Value: 4.0, Count: 99
  Value: 6.0, Count: 126
  Value: 8.0, Count: 90
  Value: 10.0, Count: 40
  Value: 12.0, Count: 33
  Value: 14.0, Count: 58
  Value: 16.0, Count: 47
  Value: 18.0, Count: 18
  Value: 20.0, Count: 23
  Value: 22.0, Count: 27
  Value: 24.0, Count: 18
  Value: 26.0, Count: 8
  Value: 28.0, Count: 9
  Value: 30.0, Count: 5
  Value: 32.0, Count: 2
  Value: 34.0, Count: 8
  Value: 36.0, Count: 6
  Value: 40.0, Count: 3
  Value: 42.0, Count: 4
  Value: 44.0, Count: 6
  Value: 46.0, Count: 7
  Value: 48.0, Count: 3
  Value: 50.0, Count: 5
  Value: 52.0, Count: 1
  Value: 54.0, Count: 5
  Value: 56.0, Count: 2
  Value: 58.0, Count: 2
  Value: 62.0, Count: 1
  Value: 64.0, Count: 1
  Value: 66.0, Count: 1
  Value: 68.0, Count: 1
  Value: 70.0, Count: 1
  Value: 80.0, Count: 1
  Value: 84.0, Count: 2
  Value: 90.0, Count: 2
  Value: 98.0, Count: 1
  Value: 104.0, Count: 2
  Value: 128.0, Count: 1
  Value: 148.0

In [24]:
unique_values, counts = np.unique(ans, return_counts=True)
print(dict(zip(unique_values, counts)))

NameError: name 'ans' is not defined

In [ ]:
array = np.full(len(data_test.x), "tp")
for i in gt:
    array[i] = 'fn'


In [ ]:
data_test.edge_index[0].unique()

In [131]:
for index, element in enumerate(ttp.test_data.test_mask.tolist()):
    if element:
        print(index)

0
3
24
25
26
27
32
36
37
47
62
66
67
68
69
70
71
72
74
75
79
102
103
104
105
106
111
115
120
122
130
164
166
170
172
174
196
198
201
203
204
205
206
219
220
221
224
225
226
227
326
328
330
331
332
351
352
359
364
365
366
367
370
371
372
373
374
375
381
389
391
395
415
459
491
526
528
529
530
531
532
535
536
537
538
539
540
541
542
543
548
552
553
632
750
754
758
759
763
780
917
925
928
929
934
937
942
1000
1001
1002
1003
1004
1005
1006
1007
1053
1054
1055
1056
1062
1063
1064
1067
1068
1073
1074
1075
1076
1079
1080
1081
1082
1085
1086
1087
1100
1101
1103
1104
1107
1108
1110
1111
1117
1119
1120
1121
1145
1223
1224
1225
1226
1227
1240
1247
1248
1249
1251
1252
1254
1255
1256
1260
1262
1263
1264
1269
1270
1271
1274
1275
1276
1278
1279
1283
1285
1286
1287
1292
1293
1295
1297
1298
1299
1302
1303
1305
1308
1309
1310
1315
1316
1317
1321
1322
1325
1331
1334
1335
1336
1339
1343
1344
1348
1349
1353
1354
1356
1357
1358
1359
1365
1368
1370
1372
1373
1380
1382
1384
1386
1387
1389
1393
1395
1397
1400


In [30]:
print(len(tt_mapped_atk_nodes_decremented))

12857


In [49]:
print(tt_mapped_atk_nodes_decremented)

2023-06-08 09:07:56
[6, 8, 10, 11, 49, 124, 181, 342, 262527, 760, 262939, 262940, 263726, 264203, 264774, 264820, 265491, 265780, 265811, 266207, 266238, 266586, 266590, 103054, 267028, 267196, 267798, 268388, 268427, 268629, 269035, 269151, 269152, 269153, 269154, 269155, 269156, 269331, 269332, 269333, 269334, 269335, 269336, 269337, 269338, 269339, 269340, 269341, 269342, 269343, 269344, 269345, 269346, 269347, 269348, 269349, 269350, 269351, 269352, 269353, 269354, 269355, 269356, 269357, 269358, 269359, 269360, 269361, 269362, 269363, 269364, 269365, 269366, 269367, 269368, 269369, 269370, 269371, 269372, 269373, 269374, 269375, 269376, 269377, 269378, 269379, 269380, 269381, 269382, 269383, 269384, 269385, 269386, 269387, 269388, 269389, 269390, 269391, 269392, 269393, 269394, 269395, 269396, 269397, 269398, 269399, 269400, 269401, 269402, 269403, 269404, 269405, 269406, 269407, 269408, 269409, 269410, 269411, 269412, 269413, 269414, 269415, 269416, 269417, 269418, 269419, 26942

In [48]:
value_max = 0 
for i in tt_mapped_atk_nodes:
	i_as_list = [i]
	hit_neighbors, edge_index, mapping, edge_mask = k_hop_subgraph(i_as_list, num_hops=2, edge_index=data_test.edge_index)
	if len(hit_neighbors) > value_max:
		value_max = len(hit_neighbors)
		print(f"max_value: {value_max}")
		print(f"nodeid: {i}")
		print(f"len of neighbors: {len(hit_neighbors)}") #first 5 entrys have a incredible high number of neighbors 32k, 3k, 26k, 40k, 6k
	# print(f"nodeid: {i}")
	# print(":")
	# print(f"len of neighbors: {len(hit_neighbors)}") #first 5 entrys have a incredible high number of neighbors 32k, 3k, 26k, 40k, 6k

max_value: 40426
nodeid: 7
len of neighbors: 40426


In [18]:
data_test.edge_index.shape 

2023-06-08 13:54:02


torch.Size([2, 7375768])

In [22]:
from torch_geometric.utils import k_hop_subgraph

In [31]:
i_as_list = [25]

hit_neighbors, edge_index, mapping, edge_mask = k_hop_subgraph(i_as_list, num_hops=2, edge_index=tt_data_test.edge_index, flow="target_to_source")
print(len(hit_neighbors))

214329


In [126]:
hit_neighbors.tolist()

[2,
 3,
 14,
 24,
 25,
 36,
 45,
 70,
 103,
 194,
 223,
 227,
 281,
 290,
 291,
 331,
 388,
 392,
 405,
 506,
 511,
 512,
 529,
 551,
 553,
 680,
 685,
 686,
 830,
 835,
 836,
 925,
 992,
 997,
 998,
 1054,
 1055,
 1100,
 1107,
 1117,
 1197,
 1202,
 1203,
 1223,
 1228,
 1250,
 1275,
 1292,
 1302,
 1324,
 1338,
 1356,
 1386,
 1521,
 1526,
 1527,
 1544,
 1570,
 1572,
 1590,
 1737,
 1742,
 1743,
 1869,
 1870,
 1900,
 1905,
 1906,
 1922,
 1944,
 1946,
 2092,
 2097,
 2098,
 2231,
 2236,
 2237,
 2342,
 2364,
 2366,
 2399,
 2404,
 2405,
 2519,
 2545,
 2547,
 2558,
 2560,
 2574,
 2579,
 2580,
 2618,
 2687,
 2694,
 2746,
 2750,
 2758,
 2764,
 2770,
 2784,
 2793,
 2813,
 2819,
 2838,
 2868,
 2873,
 2874,
 3032,
 3040,
 3041,
 3103,
 3138,
 3144,
 3155,
 3167,
 3236,
 3241,
 3242,
 3278,
 3282,
 3294,
 3300,
 3313,
 3323,
 3339,
 3410,
 3414,
 3463,
 3467,
 3505,
 3510,
 3511,
 3540,
 3564,
 3566,
 3606,
 3670,
 3676,
 3717,
 3722,
 3723,
 3743,
 3747,
 3757,
 3763,
 3777,
 3786,
 3800,
 3821,
 3

In [127]:
len(hit_neighbors)

12322

In [123]:
len(tt_mapped_atk_nodes_decremented)

12857

In [117]:
def compare_lists(list1, list2):
    set1 = set(list1)
    set2 = set(list2)
    intersection = set1.intersection(set2)
    
    # print("Length of Intersection:", len(intersection))
    # print("Intersection Elements:", intersection)
    return intersection


In [ ]:
value_max = 0 
#for i in tt_mapped_atk_nodes_decremented:
for i in tqdm(range(len(ttp.test_data.test_mask.tolist()))):
    node = tt_mapped_atk_nodes_decremented[i]
    i_as_list = [node]
    hit_neighbors, edge_index, mapping, edge_mask = k_hop_subgraph(i_as_list, num_hops=2, edge_index=tt_data_test.edge_index)
    intersection = compare_lists(hit_neighbors.tolist(), tt_mapped_atk_nodes_decremented)
    if len(intersection) > value_max:
        print(f"nodeid: {node}")
        value_max = len(intersection)
        print(f"max_value: {value_max}")
        print(f"intersection: {intersection}")
        print(f"len of neighbors: {len(intersection)}")

  0%|          | 8/357174 [00:00<2:33:17, 38.83it/s]

nodeid: 6
max_value: 1
intersection: {6}
len of neighbors: 1
nodeid: 8
max_value: 14
intersection: {266590, 269156, 8, 264203, 268427, 269035, 263726, 265491, 264820, 268629, 282388, 267196, 266238, 269151}
len of neighbors: 14


  1%|          | 2011/357174 [00:37<1:49:57, 53.83it/s]

: 

: 

In [109]:
edge_index = torch.tensor([[0, 1, 2, 3, 4, 5],
                            [2, 2, 4, 4, 6, 6]])

In [115]:
# Center node 6, 2-hops
subset, edge_index, mapping, edge_mask = k_hop_subgraph(6, 2, edge_index, directed=False)
print(subset)

print(edge_index)
print(mapping)

print(edge_mask)
print(subset[mapping])


tensor([6])
tensor([], size=(2, 0), dtype=torch.int64)
tensor([0])
tensor([False, False, False, False, False, False, False, False])
tensor([6])
